In [1]:
import json
from typing import List, Optional, Union, Literal, Dict, Any
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
from langgraph.graph import MessagesState
from langgraph.checkpoint.memory import MemorySaver
import os
import streamlit as st
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
import os

In [ ]:
# Initialize LLM
os.environ["GROQ_API_KEY"] = ""
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0
)

# Create a debug wrapper for LLM
def debug_llm_invoke(messages, **kwargs):
    print("DEBUG - LLM Input Messages:")
    for i, msg in enumerate(messages):
        print(f"Message {i} ({msg.type}):")
        print(f"{msg.content[:200]}..." if len(msg.content) > 200 else msg.content)
    
    result = llm.invoke(messages, **kwargs)
    
    print("\nDEBUG - LLM Response:")
    print(f"{result.content[:200]}..." if len(result.content) > 200 else result.content)
    return result

In [3]:
# class ValidationResult(BaseModel):
#     """Structure for response validation results"""
#     valid: bool
#     processed_value: Optional[str]
#     reasoning: str
#     follow_up_question: Optional[str] = None


class ValidationResult(BaseModel):
    """Structure for response validation results"""
    valid: bool
    processed_value: Optional[Dict[str, Optional[str]]] = None  # Changed to dictionary
    reasoning: str
    follow_up_question: Optional[str] = None
   


class ManagerDecision(BaseModel):
    """Structure for LLM's decision about next steps"""
    action: Literal["planning", "collection", "end", "general_conversation"]
    status: Literal["pending", "in_progress", "completed"]
    next_step: Optional[str] = None
    next_question: Optional[str] = None
    reasoning: str
    validation_feedback: Optional[str] = None

# State Management Classes
class CampaignStep(BaseModel):
    """Represents a single step in the campaign creation process"""
    task: str
    required_info: List[str]
    collected_info: Dict[str, Any] = Field(default_factory=dict)
    validation_rules: Dict[str, str]
    questions: Dict[str, str]  # Maps required_info to specific questions
    last_question: Optional[str] = None  # Tracks the last question asked
    expected_input: Optional[str] = None  # Tracks what input we're expecting
    status: str = "pending"  # pending/in_progress/completed
    output: Optional[str] = None

class CampaignInfo(BaseModel):
    """Tracks overall campaign information and progress"""
    steps: Dict[str, CampaignStep]
    current_step: str
    stage: str = "planning"  # planning/collection/validation/complete

class TaskState(MessagesState):
    """Main state management for the workflow"""
    conversation_id: str
    action: Optional[str]
    campaign_info: Optional[CampaignInfo]
    output: str = ""
    status: str = "pending"

# Task Identification Model
class TaskIdentification(BaseModel):
    task_type: Literal["general_convo", "campaign_convo", "other_services"]
    description: str

In [4]:
def task_identifier(state: TaskState):
    """Identifies the type of task from user input using only the last 3 messages."""
    print("\n>>> DEBUGGING TASK_IDENTIFIER NODE <<<")
    
    structured_llm = llm.with_structured_output(TaskIdentification)
    
    # Get the last 3 messages
    last_3_messages = state["messages"][-3:] if state["messages"] else []
    
    # Extract the content of the last 3 messages
    last_3_messages_content = [msg.content for msg in last_3_messages]
    
    # Combine the last 3 messages into a single string
    combined_messages = "\n".join(last_3_messages_content)
    print("Input messages:", combined_messages)
    
    # Identify task type using only the last 3 messages
    prompt = f"Identify if this is a campaign-related request or general conversation based on the following messages:\n{combined_messages}"
    print(f"Prompt to LLM: {prompt}")
    
    result = structured_llm.invoke(prompt)
    print("Task identification result:", result)
    
    output = {
        "action": result.task_type,
        "output": f"Task identified as: {result.task_type}"
    }
    print("Node output:", output)
    return output

In [5]:
def campaign_manager(state: TaskState):
    """Enhanced campaign manager with debugging"""
    print("\n>>> DEBUGGING CAMPAIGN_MANAGER NODE <<<")
    print(f"Input state action: {state['action']}")
    print(f"Input state status: {state.get('status')}")
    
    # Existing initial checks remain the same
    if state["action"] == "general_convo":
        print("Identified as general conversation, routing accordingly")
        return {"action": "general_convo"}
    
    campaign_info = state.get("campaign_info")
    print(f"Campaign info exists: {campaign_info is not None}")
    
    if not campaign_info:
        print("No campaign info, routing to planning")
        return {
            "action": "planning",
            "status": "pending"
        }

    current_step = campaign_info.steps[campaign_info.current_step]
    print(f"Current step: {campaign_info.current_step}")
    print(f"Current step status: {current_step.status}")
    
    last_message = state["messages"][-1].content if state["messages"] else None
    print(f"Last message: {last_message}")


    # If we're in waiting status, process the user's response
    if current_step.status == "waiting" and last_message:
        print("Processing user response for waiting state")

        # Get current missing information
        missing_info = [
            info for info in current_step.required_info
            if info not in current_step.collected_info
        ]

        print("---------",current_step.collected_info)

        print("MISIIIIIIING",missing_info)
        
        # Create a structured prompt for validation
        validation_prompt = f"""
        You are validating a user response for a marketing campaign setup.

        

        Context:
        Extract and validate ONLY FROM THIS RESPONSE: "{last_message}"
        Already collected: {current_step.collected_info}
        Missing fields: {missing_info}
        Validation Rules:{json.dumps(current_step.validation_rules, indent=2)}

        If the response is invalid, generate a follow-up question that:
        1. Acknowledges their response
        2. Explains what was missing or incorrect
        3. Asks for the information in a clearer way

        Return a JSON structure with these exact fields:
        {{
            "valid": boolean,
            "processed_value": extracted values for ALL fields,
            "reasoning": string explaining the validation result,
            "follow_up_question": string if invalid, null if valid
        }}
        """
        print("Validation prompt:", validation_prompt[:200] + "...")


       

        # Get LLM to validate with structured output
        structured_llm = llm.with_structured_output(ValidationResult)
        validation_result = structured_llm.invoke(validation_prompt)
        print(f"Validation result: valid={validation_result.valid}, reasoning={validation_result.reasoning[:100]}...")


# For partial validation - when some fields were extracted but others are missing

        if validation_result.valid:
            print("Response validated as valid")
            # Store the processed response
            # current_step.collected_info[current_step.last_question] = validation_result.processed_value
            current_step.collected_info.update(validation_result.processed_value)
            print(f"Updated collected_info: {current_step.collected_info}")
 
            # Check if we have all required info for this step
            missing_info = [
                info for info in current_step.required_info
                if info not in current_step.collected_info
            ]
            print(f"Missing info for current step: {missing_info}")

            if not missing_info:
                print("Step completed, looking for next incomplete step")
                # Current step is complete
                current_step.status = "completed"

                # Look for the next incomplete step
                next_incomplete_step = None
                for step_name, step in campaign_info.steps.items():
                    # Skip steps we've already completed and the current step
                    if step_name == campaign_info.current_step:
                        continue
                    if step.status != "completed":
                        next_incomplete_step = step_name
                        break

                if next_incomplete_step:
                    print(f"Found next incomplete step: {next_incomplete_step}")
                    # We found another step that needs completion
                    campaign_info.current_step = next_incomplete_step
                    campaign_info.steps[next_incomplete_step].status = "in_progress"

                    output = {
                        "action": "collection",
                        "status": "in_progress",
                        "campaign_info": campaign_info
                    }
                    print("Node output:", output)
                    return output
                else:
                    print("All steps completed, finishing campaign")
                    # All steps are completed - campaign is done
                    output = {
                        "action": "end",
                        "status": "completed",
                        "campaign_info": campaign_info,
                        "output": "Great! We've completed all the steps for your campaign setup."
                    }
                    print("Node output:", output)
                    return output
                
            else:
                print("More questions needed in this step")
                # More questions needed in this step
                current_step.status = "in_progress"
                output = {
                    "action": "collection",
                    "status": "in_progress",
                    "campaign_info": campaign_info
                }
                print("Node output:", output)
                return output
        
        else:
            print("Response validated as invalid, follow-up needed")
            # For invalid response, update status and route to END with follow-up question
            current_step.status = "waiting"  # Keep waiting since we're asking a follow-up
            # Maintain the same question we're trying to answer
            messages = state["messages"] + [validation_result.follow_up_question]
            # Return the follow-up question and route to END
            output = {
                "action": "end",
                "status": "waiting",
                "messages": messages,
                "campaign_info": campaign_info,
                "output": validation_result.follow_up_question
            }
            # print('--------------------',output)
            print("Node output action: end, with follow-up question")
            return output
    
    # Default case
    print("Continuing with current campaign state without changes")
    return state

In [7]:
def campaign_planner(state: TaskState):
    """Plans the campaign creation steps and transitions to collection stage"""
    print("\n>>> DEBUGGING CAMPAIGN_PLANNER NODE <<<")
    
    # Define the campaign creation steps with comprehensive question sets
    campaign_steps = {
        "segment_definition": CampaignStep(
            task="Define target segment",
            required_info=["segment_condition"],
            validation_rules={
                "segment_condition": "Must be a  condition in text "
            },
            questions={
                "segment_condition": "What conditions should be used to identify the target segment? (eg people who got revenue greater that 100 etc )"
            }
        ),
        "action_type": CampaignStep(
            task="Define campaign action",
            required_info=["action_type", "value", "duration"],
            validation_rules={
                "action_type": "Must be either 'bonus' or 'discount',it might be in sentence",
                "value": "Must contain a number, even if percentages, currency symbols, or names are included (e.g., 10%, $10, 10 dollars, 10 rupees)",
                "duration": "Must be a valid duration in days"
            },
           questions={
            "combined": "What type of reward (bonus/discount), value, and duration would you like to offer? "
           }
        ),
        "channel_strategy": CampaignStep(
            task="Define communication channels",
            required_info=["channels", "message_template", "frequency"],
            validation_rules={
                "channels": "Must include word like: SMS, email, push, telegram , it might be in sentence",
                "message_template": "can be any message template",
                "frequency": "Must be one of: immediate, daily, weekly"
            },
            questions={
                "channels": "Which communication channels should be used? (SMS/email/push/telegram, can select multiple)",
                "message_template": "What message should be sent to users?",
                "frequency": "How often should messages be sent? (immediate/daily/weekly)"
            }
        ),
        "scheduling": CampaignStep(
            task="Define campaign schedule",
            required_info=["start_date", "end_date", "time_zone"],
            validation_rules={
                "start_date": "can be in any date format",
                "end_date": "can be in any date format and can be in different date format than start date",
                "time_zone": "Must be a valid timezone identifier (e.g., UTC, America/New_York)"
            },
            questions={
                "start_date": "When should the campaign start? ",
                "end_date": "When should the campaign end? ",
                "time_zone": "What timezone should be used for the campaign?"
            }
        )
    }

    print(f"Setting up campaign with {len(campaign_steps)} steps")
    
    # Create campaign info with initial step
    campaign_info = CampaignInfo(
        steps=campaign_steps,
        current_step="segment_definition",
        stage="collection"  # Set initial stage to collection
    )
    
    initial_step = campaign_info.steps["segment_definition"]
    print(f"Initial step: {initial_step.task}")
    print(f"Initial step required info: {initial_step.required_info}")

    output = {
        "campaign_info": campaign_info,
        "action": "collection",
        "status": "in_progress",
        "output": "Campaign plan created. Let's start by defining the target segment."
    }
    print("Node output:", output)
    return output

In [8]:
def single_task_executor(state: TaskState):
    """Collects information for the current campaign step by generating natural questions"""
    print("\n>>> DEBUGGING SINGLE_TASK_EXECUTOR NODE <<<")
    
    campaign_info = state["campaign_info"]
    current_step = campaign_info.steps[campaign_info.current_step]
    print(f"Current step: {current_step.task}")
    print(f"Required info: {current_step.required_info}")
    print(f"Collected info: {current_step.collected_info}")

    # Get both collected and missing information
    missing_info = [
        info for info in current_step.required_info
        if info not in current_step.collected_info
    ]
    print(f"Missing info: {missing_info}")

    # Create a context section showing what we already know
    collected_context = ""
    if current_step.collected_info:
        collected_context = "Information we've already collected:\n"
        for info_key, info_value in current_step.collected_info.items():
            collected_context += f"- {info_key}: {info_value}\n"
    print(f"Collected context: {collected_context}")

    # Create a section for what we still need to know
    missing_context = "Information we still need:\n"
    for info in missing_info:
        missing_context += f"- {info}: {current_step.questions.get(info, 'No predefined question')}\n"
    print(f"Missing context: {missing_context}")

    # Generate combined question if multiple fields are needed
    if len(missing_info) > 1:
        prompt = f"""
        Generate a SINGLE question that naturally asks for all these at once:
        {', '.join([current_step.questions.get(info, info) for info in missing_info])}

        Include an example and be conversational. Ask for ALL needed information.
        """
    else:
        # Enhanced prompt that includes full context
        prompt = f"""
        You are a helpful marketing campaign assistant having a conversation with a user.
        We are currently working on the step: '{current_step.task}'

        CONTEXT OF OUR PROGRESS:
        {collected_context if collected_context else "This is the beginning of this step - no information collected yet."}

        NEXT INFORMATION NEEDED:
        We need to ask about: {missing_info[0]}
        Original question format: {current_step.questions.get(missing_info[0], 'No predefined question')}
        Validation rule: {current_step.validation_rules.get(missing_info[0], 'No validation rule')}

        TASK:
        Generate a direct, single-sentence question that:
        1. Acknowledges any previously collected information (if any exists)
        2. Clearly asks for the specific information needed
        3. Includes brief examples if necessary
        4. Must be 1-2 lines only, no lengthy explanations
     
        Keep responses under 2 sentences. Be direct and clear.
        """
    
    print(f"Prompt to LLM: {prompt[:200]}...")

    # Get next unanswered question key
    next_info_key = missing_info[0] if missing_info else None
    print(f"Next info key: {next_info_key}")

    if next_info_key:
        # Get LLM to formulate the question
        response = debug_llm_invoke([SystemMessage(content=prompt)])
        formulated_question = response.content
        print(f"Formulated question: {formulated_question}")

        # Update step status and tracking
        current_step.last_question = next_info_key
        current_step.expected_input = next_info_key
        current_step.status = "waiting"
        messages = state["messages"] + [response]

        output = {
            "campaign_info": campaign_info,
            "output": formulated_question,
            "status": "waiting",
            "messages": messages
        }
        print("Node output status: waiting with formulated question")
        return output


In [9]:
def general_conversation_agent(state: TaskState):
    """Handles general conversation with the user"""
    print("\n>>> DEBUGGING GENERAL_CONVERSATION_AGENT NODE <<<")
    
    messages = state["messages"]
    print(f"Number of input messages: {len(messages)}")
    if messages:
        print(f"Last message: {messages[-1].content[:100]}...")
    
    system_prompt = """You are a helpful assistant engaging in general conversation.
    Maintain a friendly and informative tone while providing relevant responses."""
    print(f"System prompt: {system_prompt}")

    response = debug_llm_invoke([SystemMessage(content=system_prompt)] + messages)
    print(f"Generated response: {response.content[:100]}...")

    output = {
        "output": response.content,
        "messages": messages + [response]
    }
    print("Node output: general conversation response generated")
    return output

In [10]:
def route_based_on_action(state: TaskState):
    """Routes to appropriate node based on action and status"""
    print("\n>>> DEBUGGING ROUTER <<<")
    print(f"Input state action: {state['action']}")
    print(f"Input state status: {state.get('status')}")
    
    if state["action"] == "end":
        print("Routing to END")
        return END
    elif state["action"] == "planning":
        print("Routing to campaign_planner")
        return "campaign_planner"
    elif state["action"] == "collection":
        print("Routing to single_task_executor")
        return "single_task_executor"
    elif state["action"] == "general_convo":
        print("Routing to general_conversation_agent")
        return "general_conversation_agent"
    else:
        print("Default routing to campaign_manager")
        return "campaign_manager"

In [11]:
# Build workflow graph
workflow = StateGraph(TaskState)

# Add nodes
workflow.add_node("task_identifier", task_identifier)
workflow.add_node("campaign_manager", campaign_manager)
workflow.add_node("campaign_planner", campaign_planner)
workflow.add_node("single_task_executor", single_task_executor)
workflow.add_node("general_conversation_agent", general_conversation_agent)

# Add edges
workflow.add_edge(START, "task_identifier")
workflow.add_edge("task_identifier", "campaign_manager")

# Add conditional edges
workflow.add_conditional_edges(
    "campaign_manager",
    route_based_on_action,
    {
        "campaign_planner": "campaign_planner",
        "single_task_executor": "single_task_executor",
        "general_conversation_agent": "general_conversation_agent",
        END: END
    }
)
workflow.add_edge("campaign_planner", "campaign_manager")
workflow.add_edge("single_task_executor", END)
workflow.add_edge("general_conversation_agent", END)

# Compile graph with memory
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

print("Workflow graph compiled successfully")

Workflow graph compiled successfully


In [14]:
def test_campaign_flow():
    """Test the entire campaign flow with debugging at each step"""
    print("\n===== TESTING COMPLETE CAMPAIGN FLOW =====\n")
    
    # Step 1: Initialize campaign
    config = {"configurable": {"thread_id": "test_campaign_flow"}}
    input_message = HumanMessage(content="I want to create a campaign")
    initial_state = {
        "conversation_id": "test_campaign_flow",
        "messages": [input_message],
        "action": None,
        "campaign_info": None,
        "output": "",
        "status": "pending"
    }
    
    print("STEP 1: INITIALIZING CAMPAIGN")
    res1 = graph.invoke(initial_state, config)
    print(f"\nStep 1 Output: {res1['output']}")
    
    # Step 2: Define target segment
    print("\nSTEP 2: DEFINING TARGET SEGMENT")
    segment_message = HumanMessage(content="Target users with revenue greater than $1000 in the last month")
    res1["messages"].append(segment_message)
    res2 = graph.invoke(res1, config)
    print(f"\nStep 2 Output: {res2['output']}")
    
    # Step 3: Define action type
    print("\nSTEP 3: DEFINING ACTION TYPE")
    action_message = HumanMessage(content="i would like to offer bonus")
    res2["messages"].append(action_message)
    res3 = graph.invoke(res2, config)
    print(f"\nStep 3 Output: {res3['output']}")

     # Step 3: Define action type
    print("\nSTEP 3: DEFINING ACTION TYPEfgdh")
    action_message = HumanMessage(content="10 days")
    res3["messages"].append(action_message)
    res4 = graph.invoke(res3, config)
    print(f"\nStep 3 Output: {res4['output']}")
    
    # Extract campaign info for inspection
    if res4.get("campaign_info"):
        campaign_info = res4["campaign_info"]
        current_step_name = campaign_info.current_step
        current_step = campaign_info.steps[current_step_name]
        print(f"\nCurrent campaign step: {current_step_name}")
        print(f"Collected info so far: {json.dumps(current_step.collected_info, indent=2)}")
    
    return res4

# Run the test
campaign_state = test_campaign_flow()


===== TESTING COMPLETE CAMPAIGN FLOW =====

STEP 1: INITIALIZING CAMPAIGN

>>> DEBUGGING TASK_IDENTIFIER NODE <<<
Input messages: 10 days
You mentioned '10 days', which is a valid duration. However, could you please clarify what type of action you'd like to take, such as a 'bonus' or 'discount', and what value this action will have?
I want to create a campaign
Prompt to LLM: Identify if this is a campaign-related request or general conversation based on the following messages:
10 days
You mentioned '10 days', which is a valid duration. However, could you please clarify what type of action you'd like to take, such as a 'bonus' or 'discount', and what value this action will have?
I want to create a campaign
Task identification result: task_type='campaign_convo' description='The user wants to create a campaign'
Node output: {'action': 'campaign_convo', 'output': 'Task identified as: campaign_convo'}

>>> DEBUGGING CAMPAIGN_MANAGER NODE <<<
Input state action: campaign_convo
Input state s